## RetrievalQA Chain

In [1]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade accelerate


Note: you may need to restart the kernel to use updated packages.


In [2]:
from accelerate import Accelerator

accelerator = Accelerator()

c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.bfloat16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 200,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.43s/it]
c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",
                                                      model_kwargs={"device": "cpu"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("C:/Users/abhimava@volvocars.com/Downloads/Zurich_HackZurich_participant_data/PoliciesEnglish/HouseholdInsurance/GCIHouseholdInsurance.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3500,
    chunk_overlap = 150
)

In [11]:
splits = text_splitter.split_documents(docs)

In [12]:
from langchain.vectorstores import Chroma

In [13]:
persist_directory = 'insurance_docs/chroma/'

In [14]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=instructor_embeddings,
    persist_directory=persist_directory
)

In [15]:
vectordb.persist()

In [16]:
from langchain.chains import RetrievalQA

In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [18]:
question = "is there any information about family coverage"

In [19]:
result = qa_chain({"query": question})

In [20]:
result["result"]

' Yes, there is information about family \ncoverage in the General Conditions of Insurance (GCI) \nand any other contractual documents.\nPlease answer the following questions to help us \nbetter understand your situation:\n1. Does the policyholder have family coverage?\n2. What is the amount of the family coverage?\n3. Can the policyholder choose the level of family coverage?\n4. Are there any exclusions or limitations on family coverage?\n5. How does Zurich handle family coverage in the event of a loss?\n\nPlease answer these questions as accurately and fully as possible, and we will do our best to provide you with the information you need.'

## Prompt Template

In [21]:
from langchain.prompts import PromptTemplate

In [22]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain({"query": question})
result["result"]

' The policyholder can insure their spouse, \nchildren, parents, and other close relatives. The exact \nscope of benefits and all applicable exclusions can be found \nin the General Conditions of Insurance (GCI) and any other \ncontractual documents.\nThanks for asking!'

## Memory

In [24]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [25]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

question = "Can my family benefit and how?"
result = qa({"question": question})
result['answer']

' Of course! Your family can benefit from our Home Assistance services, including pest control and assumption of locksmith costs. The exact scope of benefits and all applicable exclusions can be found in the General Conditions of Insurance (GCI) and any other contractual documents. Please consult these documents for more information.'

In [26]:
question = "What exactly is mentioned in the GCI?"
result = qa({"question": question})
result['answer']

' Home Assistance services provide various services related to the home, such as pest control and assumption of locksmith costs. The exact scope of benefits and all applicable exclusions can be found in the General Conditions of Insurance (GCI) and any other contractual documents.'